# Applied Capstone Project

## Opening New Shopping Mall in Kuala Lumpur Malaysia

**By Navin Krishna**

**June 2019**

## Introduction

Visiting shopping mall is a relaxation activity for most of family. Also wide variety of age groups should hangout in shopping malls. There are many features such as grocery shopping, dine at restaurant and shop at various fashion outlets. For retailers, the central location and large crowd at shopping malls provide great distribution channel to market their products and services.

#### Bussiness Problem

The objective of this capstone project is to anayse and select the best locations in Kuala Lumpur, Malaysia to open a new shopping mall. Using Data science methodology and machine learning techniques like clustering, this project aim to provide solution to answer the bussiness question: In the city of Kula Lumpur, Malaysia, if a property developer is looking to open a new shopping mall,where would you recommend to open it?

#### Target audience of this project

This project is useful for developers and investors looking to open or invest new shoppping mall in kula Lumpur.

# Data

- List of neighbourhood data from wikipedia(https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur)
- Latitude and Longitude coordinates of neighbourhood from geocoder
- Venue data is extracted from Foursquare using foursquare API

## Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## Scrap wikipedia data into dataframe

In [4]:
# GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [5]:
# parse data using beautifulsoup
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neigh_List = []

In [7]:
# append data into neigh_list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neigh_List.append(row.text)

In [8]:
# creating new dataframe
kul_df = pd.DataFrame({"Neighborhood": neigh_List})
kul_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [9]:
# print the number of rows of the dataframe
kul_df.shape

(71, 1)

# Get the cordinates with Geocoder

In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kul_df["Neighborhood"].tolist() ]

In [12]:
coords

[[3.0576900000000364, 101.74388000000005],
 [3.148494115588384, 101.69672876508707],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008],
 [3.147890000000075, 101.69405000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.0578100000000745, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1511418740671178, 101.69937468268016],
 [3.129290000000026, 101.69896000000006],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.1479700000000435, 101.66795000000008],
 [3.136442497863743, 101.69029641203808],
 [3.1419057131529953, 101.67967833034047],
 [3.1566851613075664, 101.6

In [13]:
# create dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
# merge the coordinates into the kul_df dataframe
kul_df['Latitude'] = df_coords['Latitude']
kul_df['Longitude'] = df_coords['Longitude']

In [15]:
# check the neighborhoods and the coordinates
print(kul_df.shape)
kul_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148494,101.696729
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.129200,101.678440
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.061870,101.746750


In [24]:
# save the DataFrame as CSV file
kul_df.to_csv("kul_df.csv", index=False)

### Create a map of Kuala Lumpur

In [16]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516964, 101.6942371.


In [26]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kul_df['Latitude'], kul_df['Longitude'], kul_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [27]:
# save the map as HTML file
map_kl.save('map_kl.html')

In [17]:
CLIENT_ID = 'MPHIY1RX3WX5BYZATQYJOAS5GXJYV1ZIPBFOH1GRJPWVJBBE' # your Foursquare ID
CLIENT_SECRET = 'IHEKQ3ZH3JIQLKO5WEFU5QE2MYMXCFYS3P4AEZJJCEP4554I' # your Foursquare Secret
VERSION = '20181201' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MPHIY1RX3WX5BYZATQYJOAS5GXJYV1ZIPBFOH1GRJPWVJBBE
CLIENT_SECRET:IHEKQ3ZH3JIQLKO5WEFU5QE2MYMXCFYS3P4AEZJJCEP4554I


In [30]:
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
results = requests.get(url).json()

In [31]:
results["response"]['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bbff1f074a9a5933d78cff6',
   'name': 'Hilton',
   'location': {'address': '3 Jalan Stesen Sentral',
    'lat': 3.135573436903536,
    'lng': 101.68577374696778,
    'distance': 89,
    'postalCode': '50470',
    'cc': 'MY',
    'city': 'Kuala Lumpur',
    'state': 'Kuala Lumpur',
    'country': 'Malaysia',
    'formattedAddress': ['3 Jalan Stesen Sentral',
     '50470 Kuala Lumpur, Kuala Lumpur',
     'Malaysia']},
   'categories': [{'id': '4bf58dd8d48988d1fa931735',
     'name': 'Hotel',
     'pluralName': 'Hotels',
     'shortName': 'Hotel',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4bbff1f074a9a5933d78cff6-0'},
 {'reasons': {'count': 0,
   'items': [{'summary': 'This spo

In [18]:
# Top 100 venues within 2000 meters radius
radius = 2000
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
kul_df = pd.read_csv("kul_df.csv")

In [25]:
kul_venues = getNearbyVenues(names=kul_df['Neighborhood'],
                                   latitudes=kul_df['Latitude'],
                                   longitudes=kul_df['Longitude']
                                  )

Alam Damai
Ampang, Kuala Lumpur
Bandar Sri Permaisuri
Bandar Tasik Selatan
Bandar Tun Razak
Bangsar
Bangsar Park
Bangsar South
Batu 11 Cheras
Batu, Kuala Lumpur
Brickfields
Bukit Bintang
Bukit Kiara
Bukit Nanas
Bukit Petaling
Bukit Tunku
Cheras, Kuala Lumpur
Chow Kit
Damansara Heights
Damansara Town Centre
Damansara, Kuala Lumpur
Dang Wangi
Desa Petaling
Federal Hill, Kuala Lumpur
Happy Garden
Jalan Cochrane, Kuala Lumpur
Jalan Duta
Jinjang
Kampung Baru, Kuala Lumpur
Kampung Datuk Keramat
Kampung Padang Balang
Kepong
Kepong Baru
KL Eco City
Kuchai Lama
Lembah Pantai
Maluri
Medan Tuanku
Miharja
Mont Kiara
Pantai Dalam
Pudu, Kuala Lumpur
Putrajaya
Salak South
Segambut
Semarak
Sentul, Kuala Lumpur
Setapak
Setiawangsa
Shamelin
Sri Hartamas
Sri Petaling
Sungai Besi
Taman Bukit Maluri
Taman Connaught
Taman Desa
Taman Ibukota
Taman Len Seng
Taman Melati
Taman Midah
Taman P. Ramlee
Taman Sri Sinar
Taman Taynton View
Taman Tun Dr Ismail
Taman U-Thant
Taman Wahyu
Titiwangsa
Wangsa Maju


In [27]:
kul_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
1,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
2,Alam Damai,3.05769,101.74388,閒茶素食店 Leisure Tea Vegetarian,3.057673,101.747258,Vegetarian / Vegan Restaurant
3,Alam Damai,3.05769,101.74388,Roti Canai Perindu,3.055071,101.740574,Breakfast Spot
4,Alam Damai,3.05769,101.74388,Heng Kee Tim Sum 興記點心,3.057513,101.747191,Dim Sum Restaurant


In [30]:
kul_venues.to_csv("kul_venues.csv")

In [29]:
kul_venues.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alam Damai,23,23,23,23,23,23
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Sri Permaisuri,41,41,41,41,41,41
Bandar Tasik Selatan,38,38,38,38,38,38
Bandar Tun Razak,9,9,9,9,9,9
Bangsar,27,27,27,27,27,27
Bangsar Park,27,27,27,27,27,27
Bangsar South,64,64,64,64,64,64
Batu 11 Cheras,39,39,39,39,39,39


In [33]:
# Unique categories
print('There are {} uniques categories.'.format(len(kul_venues['Venue Category'].unique())))

There are 276 uniques categories.


In [35]:
#list of categories
kul_venues['Venue Category'].unique()[:50]

array(['Noodle House', 'Chinese Restaurant',
       'Vegetarian / Vegan Restaurant', 'Breakfast Spot',
       'Dim Sum Restaurant', 'Dessert Shop', 'Convenience Store',
       'Farmers Market', 'Hakka Restaurant', 'Japanese Restaurant',
       'Pharmacy', 'Fast Food Restaurant', 'Food Truck',
       'Asian Restaurant', 'Food Court', 'Indian Restaurant', 'Bookstore',
       'Malay Restaurant', 'Padangnese Restaurant', 'Hostel', 'Exhibit',
       'Chettinad Restaurant', 'South Indian Restaurant',
       'Monument / Landmark', 'Park', 'Restaurant', 'Hotel', 'Gym',
       'Coffee Shop', 'Athletics & Sports', 'Juice Bar', 'Theater',
       'Café', 'Bakery', 'Flower Shop', 'Bar', 'Wine Bar', 'Art Gallery',
       'Thai Restaurant', 'Bubble Tea Shop', 'Bed & Breakfast',
       'Bridal Shop', 'Food Stand', 'Clothing Store', 'Cupcake Shop',
       'Other Nightlife', 'Arts & Crafts Store', 'Sandwich Place',
       'Filipino Restaurant', 'Flea Market'], dtype=object)

In [37]:
# check if the results contain "Shopping Mall"
"Neighborhood" in kul_venues['Venue Category'].unique()

True

In [39]:
# one hot encoding
kl_onehot = pd.get_dummies(kul_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = kul_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2607, 277)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Business Service,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Stadium,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Imperial Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Poke Place,Police Station,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Alam Damai

In [40]:
# Group the row by neighbourhood
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(68, 277)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Business Service,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Stadium,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Imperial Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Poke Place,Police Station,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Alam Damai

In [41]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

12

In [42]:
# New data frame for shopping mall data
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.0
1,"Ampang, Kuala Lumpur",0.0
2,Bandar Sri Permaisuri,0.0
3,Bandar Tasik Selatan,0.0
4,Bandar Tun Razak,0.0


In [43]:
# Cluster Neighbourhood
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0])

In [44]:
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [45]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.0,0
1,"Ampang, Kuala Lumpur",0.0,0
2,Bandar Sri Permaisuri,0.0,0
3,Bandar Tasik Selatan,0.0,0
4,Bandar Tun Razak,0.0,0


In [48]:
kl_merged = kl_merged.join(kul_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head()

(68, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.0,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.0,0,3.148494,101.696729
2,Bandar Sri Permaisuri,0.0,0,3.103910,101.712260
3,Bandar Tasik Selatan,0.0,0,3.072750,101.714610
4,Bandar Tun Razak,0.0,0,3.082800,101.722810


In [49]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(68, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.000000,0,3.057690,101.743880
35,Lembah Pantai,0.000000,0,3.121202,101.663899
36,Maluri,0.000000,0,3.147890,101.694050
37,Medan Tuanku,0.000000,0,3.159260,101.698340
38,Miharja,0.000000,0,3.147890,101.694050
40,Pantai Dalam,0.000000,0,3.094760,101.667470
41,"Pudu, Kuala Lumpur",0.000000,0,3.133540,101.713070
42,Putrajaya,0.000000,0,3.125843,101.718509
43,Salak South,0.000000,0,3.081020,101.697240
44,Segambut,0.000000,0,3.186390,101.668100


In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine clusters

In [57]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.0,0,3.057690,101.743880
35,Lembah Pantai,0.0,0,3.121202,101.663899
36,Maluri,0.0,0,3.147890,101.694050
37,Medan Tuanku,0.0,0,3.159260,101.698340
38,Miharja,0.0,0,3.147890,101.694050
40,Pantai Dalam,0.0,0,3.094760,101.667470
41,"Pudu, Kuala Lumpur",0.0,0,3.133540,101.713070
42,Putrajaya,0.0,0,3.125843,101.718509
43,Salak South,0.0,0,3.081020,101.697240
44,Segambut,0.0,0,3.186390,101.668100


In [58]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
63,Taman Tun Dr Ismail,0.030769,1,3.152830,101.622710
7,Bangsar South,0.031250,1,3.111020,101.662830
8,Batu 11 Cheras,0.025641,1,3.061870,101.746750
23,"Federal Hill, Kuala Lumpur",0.027778,1,3.136370,101.685640
16,"Cheras, Kuala Lumpur",0.025641,1,3.061870,101.746750
19,Damansara Town Centre,0.035714,1,3.136442,101.690296
67,Wangsa Maju,0.026316,1,3.203910,101.737190


In [59]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
39,Mont Kiara,0.014925,2,3.165320,101.652430
25,"Jalan Cochrane, Kuala Lumpur",0.020000,2,3.132903,101.724679
11,Bukit Bintang,0.010417,2,3.147770,101.708550
28,KL Eco City,0.020000,2,3.117130,101.673840
13,Bukit Nanas,0.010753,2,3.151142,101.699375


### Observations:

Most of the shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 and moderate number in cluster 0.